In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
matplotlib.rcParams["figure.dpi"] = 150
import ipywidgets as widgets
from particle import PDGID
import sys
sys.path.append("../../../Muon_Collider_Smart_Pixels/daniel/validationPlots")
from plotUtils import *
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes

In [2]:
def plotHits(sig,pltStandalone=True,pltShow=True,alpha = 1,s=1):
    if pltStandalone:
        fig,ax=plt.subplots()
    else:
        ax = plt.gca()

    colorsList = ["r","b","g","c","m","y","aquamarine","fuchsia",
                "sienna","deepskyblue","springgreen","royalblue",
                "darkorange","indigo","navy","chartreuse"]
    
    colors = sig['moduleID'].apply(lambda ID: colorsList[list(np.arange(1,17,1)).index(ID)])
    ax.scatter(sig['hit_x'],sig["hit_y"],s=s, c=colors, label="Signal hits",alpha = alpha)
    plt.gca().set_aspect('equal')
    circle=plt.Circle((0,0), 30, fill=0, color='k', label="Barrel")
    ax.add_patch(circle)
    ax.set_title("Hits in the barrel xy-plane")
    ax.set_xlabel("x-global [mm]")
    ax.set_ylabel("y-global [mm]")
    ax.set_xlim(-40,40)
    ax.set_ylim(-40,40)
    if pltShow:
        plt.show()

In [3]:
def plot(sig,moduleID,i,use_weird_tracks=False):
    cut1 = sig['moduleID']==moduleID
    if use_weird_tracks:
        cut2=abs(sig['p']-sig['p_from_angles'])>10
        cut = cut1 & cut2
    else:
        cut = cut1
    # Pull single point and vector
    tempsig=sig[cut].reset_index(drop=True)
    gamma=tempsig['gamma'][i]
    vec0 = tempsig['vec0'][i]
    vec1 = tempsig['vec1'][i]
    vec11 = tempsig['vec11'][i]
    x0 = tempsig['hit_x'][i]
    y0 = tempsig['hit_y'][i]
    y1 = tempsig['ylocal'][i]

    fig,ax=plt.subplots(ncols=2)
    ax[0].scatter(sig['hit_x'][cut1],sig['hit_y'][cut1],s=1, c='r')
    ax[0].scatter(x0,y0,c='b')
    ax[0].quiver(x0,y0,vec0[0],vec0[1],scale=10)
    ax[0].quiver(0,0,np.cos(gamma),np.sin(gamma),scale=10)
    plt.gca().set_aspect('equal')
    circle=plt.Circle((0,0), 30, fill=0, color='k')
    ax[0].add_patch(circle)
    ax[0].set_title("BIB hit location in xy-plane")
    ax[0].set_xlabel("x [mm]")
    ax[0].set_ylabel("y [mm]")
    ax[0].set_xlim(-40,40)
    ax[0].set_ylim(-40,40)
    ax[0].set_aspect('equal')

    ax[1].scatter(y1,0,c='b')
    ax[1].quiver(y1,0,vec1[0],vec1[1],scale=10,color="k")
    ax[1].quiver(y1,0,vec11[0],vec11[1],scale=10,color="b")
    ax[1].hlines(y=0,color='r',xmin=-4.5,xmax=8.5)
    ax[1].set_xlim(-5.5, 9.5) 
    ax[1].set_ylim(-1, 2) 
    ax[1].invert_xaxis()

    ax[1].set_title(f'Total Range: {y1}; ylocal={sig["ylocal"][i]}; Gamma={gamma}')

    plt.show()


In [4]:
sig = pd.read_csv('/local/d1/smartpixML/2026Datasets/Data_Files/Data_Set_flp_0/Track_Lists/signal_tracks_extra_info_0.txt', sep=' ')

In [5]:
max(sig['ylocal'])-min(sig['ylocal'])

13.0

In [23]:
sig['vec0']=list(zip(np.cos(sig['phi']), np.sin(sig['phi'])))
sig['vec0']=list(zip(np.cos(sig['phi']), np.sin(sig['phi'])))
sig['nz']=1/np.sqrt(1+sig['cota']**2+sig['cotb']**2)
sig['ny']=sig['cotb']*sig['nz']
sig['nx']=sig['cota']*sig['nz']
sig['vec1']=list(zip(sig['ny']/np.sqrt(sig['nz']**2+sig['ny']**2), sig['nz']/np.sqrt(sig['nz']**2+sig['ny']**2)))
sig['p_from_angles']=sig['pt'] / np.sqrt((sig['nz']**2 + sig['ny']**2))


In [28]:
print(len(sig[abs(sig['p']-sig['p_from_angles'])>1]))

0


In [8]:
sensorAngles = np.arange(-np.pi-np.pi/8,np.pi+2*np.pi/8,np.pi/8)
moduleIDs = np.concatenate((np.arange(8,17,1), np.arange(1,11,1)))
# CHECK THAT THIS IS CORRECT!

def getYlocalAndGamma(x,y):
    # Get exact angle gamma of the hit position
    gammaP=np.arctan2(y,x)

    # Get two sensor angles that are closest to the exact angle
    diff = np.abs(sensorAngles-gammaP)
    index1 = np.argmin(diff)
    gamma1=sensorAngles[index1]
    diff[index1]=3*np.pi # reassign the previous min to something much larger than the other values
    index2 = np.argmin(diff)
    gamma2=sensorAngles[index2]

    # Rotate x coordinate of the point by each option for gamma
    x1=x*np.cos(-gamma1)-y*np.sin(-gamma1)
    y1=y*np.cos(-gamma1)+x*np.sin(-gamma1)
    x2=x*np.cos(-gamma2)-y*np.sin(-gamma2)
    y2=y*np.cos(-gamma2)+x*np.sin(-gamma2)

    print(x1,y1)
    print(x2,y2)

    # Determine which x is closest to expected value
    xTrue=30.16475324197002

    diff1=abs(x1-xTrue)
    diff2=abs(x2-xTrue)
    
    # If both x1 and x2 are really close to the correct value
    if diff1 < 0.5 and diff2 < 0.5:
        if y1>8.5 or y1<-4.5:
            index=index2
        else:
            index=index1
            
    elif diff1<diff2:
        index=index1
    else:
        index=index2

    if index==index1:
        yentry=y1
    else:
        yentry=y2
    
    ylocal=round(yentry/25e-3)*25e-3
    # at some point, add limits to possible ROIs

    gamma=sensorAngles[index]
    moduleID=moduleIDs[index]

    print(gamma,moduleID)

In [43]:
def plot(moduleID,i,):
    cut = sig['moduleID']==moduleID

    # Pull single point and vector
    tempsig=sig[cut].reset_index(drop=True)
    gamma=tempsig['gamma'][i]
    vec0 = tempsig['vec0'][i]
    vec1 = tempsig['vec1'][i]
    x0 = tempsig['hit_x'][i]
    y0 = tempsig['hit_y'][i]
    y1 = tempsig['ylocal'][i]
    flp=tempsig['flp'][i]

    fig,ax=plt.subplots(ncols=2)
    ax[0].scatter(sig['hit_x'][cut],sig['hit_y'][cut],s=1, c='r')
    ax[0].scatter(x0,y0,c='b')
    ax[0].quiver(x0,y0,vec0[0],vec0[1],scale=10)
    plt.gca().set_aspect('equal')
    circle=plt.Circle((0,0), 30, fill=0, color='k')
    ax[0].add_patch(circle)
    ax[0].set_title("BIB hit location in xy-plane")
    ax[0].set_xlabel("x [mm]")
    ax[0].set_ylabel("y [mm]")
    ax[0].set_xlim(-40,40)
    ax[0].set_ylim(-40,40)
    ax[0].set_aspect('equal')

    ax[1].scatter(y1,0,c='b')
    ax[1].quiver(y1,0,vec1[0],vec1[1],scale=10,color="k")
    if flp==0:
        ax[1].hlines(y=0,color='r',xmin=4.5,xmax=-8.5)
        ax[1].set_xlim(-9.5,5.5) 
    else:
        ax[1].hlines(y=0,color='r',xmin=-4.5,xmax=8.5)
        ax[1].set_xlim(9.5,-5.5) 
    ax[1].set_ylim(-1, 2) 
    ax[1].set_title(f'ylocal={sig["ylocal"][i]}; Gamma={gamma}')

    plt.show()

In [44]:
widgets.interact(plot, moduleID=widgets.IntSlider(min=1, max=16, step=1), i=widgets.IntSlider(min=0, max=100, step=1))

interactive(children=(IntSlider(value=1, description='moduleID', max=16, min=1), IntSlider(value=0, descriptio…

<function __main__.plot(moduleID, i)>